In [1]:
import csv
from itertools import islice
import json
import os
from pkg_resources import resource_filename

from nupic.data.file_record_stream import FileRecordStream
from nupic.engine import Network
from nupic.encoders import MultiEncoder, ScalarEncoder, DateEncoder

In [2]:
_VERBOSITY = 0

# Default config fields for SPRegion
_SP_PARAMS = {
    "spVerbosity": _VERBOSITY,
    "spatialImp": "cpp",
    "globalInhibition": 1,
    "columnCount": 2048,
    "inputWidth": 0,
    "numActiveColumnsPerInhArea": 40,
    "seed": 1956,
    "potentialPct": 0.8,
    "synPermConnected": 0.1,
    "synPermActiveInc": 0.0001,
    "synPermInactiveDec": 0.0005,
    "boostStrength": 0.0,
}

# Default config fields for TPRegion
_TM_PARAMS = {
    "verbosity": _VERBOSITY,
    "columnCount": 2048,
    "cellsPerColumn": 32,
    "inputWidth": 2048,
    "seed": 1960,
    "temporalImp": "cpp",
    "newSynapseCount": 20,
    "maxSynapsesPerSegment": 32,
    "maxSegmentsPerCell": 128,
    "initialPerm": 0.21,
    "permanenceInc": 0.1,
    "permanenceDec": 0.1,
    "globalDecay": 0.0,
    "maxAge": 0,
    "minThreshold": 9,
    "activationThreshold": 12,
    "outputType": "normal",
    "pamLength": 3,
}

_NUM_RECORDS = 92

In [3]:
def createTemporalAnomaly(recordParams, spatialParams=_SP_PARAMS,
                          temporalParams=_TM_PARAMS,
                          verbosity=_VERBOSITY):

    """Generates a Network with connected RecordSensor, SP, TM.
    This function takes care of generating regions and the canonical links.
    The network has a sensor region reading data from a specified input and
    passing the encoded representation to an SPRegion.
    The SPRegion output is passed to a TMRegion.
    Note: this function returns a network that needs to be initialized. This
    allows the user to extend the network by adding further regions and
    connections.
    :param recordParams: a dict with parameters for creating RecordSensor region.
    :param spatialParams: a dict with parameters for creating SPRegion.
    :param temporalParams: a dict with parameters for creating TMRegion.
    :param verbosity: an integer representing how chatty the network will be.
    """
    inputFilePath = recordParams["inputFilePath"]
    scalarEncoder1Args = recordParams["scalarEncoder1Args"]
    scalarEncoder2Args = recordParams["scalarEncoder2Args"]
    scalarEncoder3Args = recordParams["scalarEncoder3Args"]
    dateEncoderArgs = recordParams["dateEncoderArgs"]

    scalarEncoder1 = ScalarEncoder(**scalarEncoder1Args)
    scalarEncoder2 = ScalarEncoder(**scalarEncoder2Args)
    scalarEncoder3 = ScalarEncoder(**scalarEncoder3Args)
    dateEncoder = DateEncoder(**dateEncoderArgs)

    encoder = MultiEncoder()
    encoder.addEncoder(scalarEncoder1Args["name"], scalarEncoder1)
    encoder.addEncoder(scalarEncoder2Args["name"], scalarEncoder2)
    encoder.addEncoder(scalarEncoder3Args["name"], scalarEncoder3)
    encoder.addEncoder(dateEncoderArgs["name"], dateEncoder)

    network = Network()

    network.addRegion("sensor", "py.RecordSensor",
                    json.dumps({"verbosity": verbosity}))

    sensor = network.regions["sensor"].getSelf()
    sensor.encoder = encoder
    sensor.dataSource = FileRecordStream(streamID=inputFilePath)

    # Create the spatial pooler region
    spatialParams["inputWidth"] = sensor.encoder.getWidth()
    network.addRegion("spatialPoolerRegion", "py.SPRegion",
                      json.dumps(spatialParams))

    # Link the SP region to the sensor input
    network.link("sensor", "spatialPoolerRegion", "UniformLink", "")
    network.link("sensor", "spatialPoolerRegion", "UniformLink", "",
                 srcOutput="resetOut", destInput="resetIn")
    network.link("spatialPoolerRegion", "sensor", "UniformLink", "",
                 srcOutput="spatialTopDownOut", destInput="spatialTopDownIn")
    network.link("spatialPoolerRegion", "sensor", "UniformLink", "",
                 srcOutput="temporalTopDownOut", destInput="temporalTopDownIn")

    # Add the TPRegion on top of the SPRegion
    network.addRegion("temporalPoolerRegion", "py.TMRegion",
                      json.dumps(temporalParams))

    network.link("spatialPoolerRegion", "temporalPoolerRegion", "UniformLink", "")
    network.link("temporalPoolerRegion", "spatialPoolerRegion", "UniformLink", "",
                 srcOutput="topDownOut", destInput="topDownIn")

    spatialPoolerRegion = network.regions["spatialPoolerRegion"]

    # Make sure learning is enabled
    spatialPoolerRegion.setParameter("learningMode", True)
    # We want temporal anomalies so disable anomalyMode in the SP. This mode is
    # used for computing anomalies in a non-temporal model.
    spatialPoolerRegion.setParameter("anomalyMode", False)

    temporalPoolerRegion = network.regions["temporalPoolerRegion"]

    # Enable topDownMode to get the predicted columns output
    temporalPoolerRegion.setParameter("topDownMode", True)
    # Make sure learning is enabled (this is the default)
    temporalPoolerRegion.setParameter("learningMode", True)
    # Enable inference mode so we get predictions
    temporalPoolerRegion.setParameter("inferenceMode", True)
    # Enable anomalyMode to compute the anomaly score.
    temporalPoolerRegion.setParameter("anomalyMode", True)

    return network

def getDate(recordParams, total = _NUM_RECORDS):
    inputFilePath = recordParams["inputFilePath"]
    date = []
    with open(inputFilePath) as fin:
        reader = csv.reader(fin)
        headers = reader.next()  # skip the header
        reader.next()
        reader.next()
        for record in islice(reader, total):
            Record_in_Dict = dict(zip(headers, record))
            date.append(Record_in_Dict["time"])
    return date
    
def runNetwork(network, date):
    """Run the network and write output to writer.
    :param network: a Network instance to run
    :param writer: a csv.writer instance to write output to
    """
    sensorRegion = network.regions["sensor"]
    temporalPoolerRegion = network.regions["temporalPoolerRegion"]

    for i in xrange(_NUM_RECORDS):
      # Run the network for a single iteration
        network.run(1)
      # Write out the anomaly score along with the record number and consumption
      # value.
        anomalyScore = temporalPoolerRegion.getOutputData("anomalyScore")[0]
        pre1 = sensorRegion.getOutputData("sourceOut")[0]
        pre2 = sensorRegion.getOutputData("sourceOut")[1]
        pre3 = sensorRegion.getOutputData("sourceOut")[2]
        print "date: ", date[i], "\t Anomaly Score: ", anomalyScore, "\t pepa_acc: ", (pre1, pre2, pre3)

In [4]:
if __name__ == "__main__":
    inputFilePath = '/mnt/data/project/HTM-AnomalyDetection/data/pepa_acc_fake.csv'

    scalarEncoder1Args = {
      "w": 21,
      "minval": -2.0,
      "maxval": 2.0,
      "periodic": False,
      "n": 50,
      "radius": 0,
      "resolution": 0,
      "name": "pepa_acc_x",
      "verbosity": 0,
      "clipInput": True,
      "forced": False,
    }
    
    scalarEncoder2Args = {
      "w": 21,
      "minval": -2.0,
      "maxval": 2.0,
      "periodic": False,
      "n": 50,
      "radius": 0,
      "resolution": 0,
      "name": "pepa_acc_y",
      "verbosity": 0,
      "clipInput": True,
      "forced": False,
    }

    scalarEncoder3Args = {
      "w": 21,
      "minval": -2.0,
      "maxval": 2.0,
      "periodic": False,
      "n": 50,
      "radius": 0,
      "resolution": 0,
      "name": "pepa_acc_z",
      "verbosity": 0,
      "clipInput": True,
      "forced": False,
    }
    
    dateEncoderArgs = {
      "season": 0,
      "dayOfWeek": 0,
      "weekend": 0,
      "holiday": 0,
      "timeOfDay": (21, 1),
      "customDays": 0,
      "name": "time",
      "forced": False
    }

    recordParams = {
      "inputFilePath": inputFilePath,
      "scalarEncoder1Args": scalarEncoder1Args,
      "scalarEncoder2Args": scalarEncoder2Args,
      "scalarEncoder3Args": scalarEncoder3Args,
      "dateEncoderArgs": dateEncoderArgs,
    }

    network = createTemporalAnomaly(recordParams)
    date = getDate(recordParams, _NUM_RECORDS)
    runNetwork(network, date)

date:  2018-10-25 21:00:00 	 Anomaly Score:  1.0 	 pepa_acc:  (0.98393428, -0.0055931741, -0.015456213)
date:  2018-10-25 21:00:01 	 Anomaly Score:  1.0 	 pepa_acc:  (0.98507893, -0.021764981, -0.013305826)
date:  2018-10-25 21:00:02 	 Anomaly Score:  1.0 	 pepa_acc:  (0.98496461, -0.010979516, -0.01410551)
date:  2018-10-25 21:00:03 	 Anomaly Score:  0.0 	 pepa_acc:  (0.97859609, -0.0071278112, -0.015474239)
date:  2018-10-25 21:00:04 	 Anomaly Score:  0.0 	 pepa_acc:  (0.98197186, -0.0041466579, -0.015434567)
date:  2018-10-25 21:00:05 	 Anomaly Score:  0.0 	 pepa_acc:  (0.98057491, -0.01003095, -0.017894926)
date:  2018-10-25 21:00:06 	 Anomaly Score:  0.0 	 pepa_acc:  (0.98349404, -0.010017129, -0.014090998)
date:  2018-10-25 21:00:07 	 Anomaly Score:  0.0 	 pepa_acc:  (0.98402172, -0.013944687, -0.011768029)
date:  2018-10-25 21:00:08 	 Anomaly Score:  0.0 	 pepa_acc:  (0.98734689, -0.0060340231, -0.016891809)
date:  2018-10-25 21:00:09 	 Anomaly Score:  0.0 	 pepa_acc:  (0.984923